In [0]:
# solo en bronce ,ya que en pipeline se hace con parametro general
dbutils.widgets.text("path", "", "")
path = dbutils.widgets.get("path")

In [0]:
# Silver, limpieza y transformacion de datos
from pyspark.sql.window import Window
from pyspark.sql.functions import *

df_bankloans_plata = spark.read.format("delta").load(f"/{path}/bronce/delta_tables")

df_bankloans_plata = df_bankloans_plata.drop("Zip_Code")

df_bankloans_plata = df_bankloans_plata.filter((col("Experience") >= 0) & (col("Experience") <= col("Age")))

df_bankloans_plata = df_bankloans_plata \
    .withColumn("income", col("Income").cast("double"))

df_bankloans_plata = df_bankloans_plata.withColumn("Mortgage", col("Mortgage").cast("double"))


# Convertir CC_Avg a valor numérico
df_bankloans_plata = df_bankloans_plata.withColumn(
    "CC_Avg",
    when(col("CC_Avg").contains("/"),  
         (split(col("CC_Avg"), "/").getItem(0).cast("float") /
          split(col("CC_Avg"), "/").getItem(1).cast("float"))
    )
)

df_bankloans_plata = df_bankloans_plata.withColumn("Education",
                   when(col("Education") == 1, "Undergrad")
                   .when(col("Education") == 2, "Graduate")
                   .when(col("Education") == 3, "Advanced/Professional")
                   .otherwise("Unknown"))

# Corregir valores negativos en Experience (podrías establecerlos a 0 o imputar basado en edad)
df_bankloans_plata = df_bankloans_plata.withColumn(
    'Experience',
    when(col('Experience') < 0, 0).otherwise(col('Experience'))
)

display(df_bankloans_plata)


In [0]:
# silver save Csv and Delta
df_bankloans_plata.write.format("csv").option("header","true").mode("overwrite").save(f"/{path}/plata/csv")

df_bankloans_plata.write.format("delta").mode("overwrite").save(f"/{path}/plata/delta_tables")
